# Año 2012

## Configuración

In [11]:
# Extracción de datos 
# --- 1. SISTEMA Y SEGURIDAD ---
# Gestiona variables de entorno y control de pausas
from dotenv import load_dotenv       # Carga el archivo .env con claves secretas
load_dotenv()                        # Inyecta las claves como variables de entorno
import os                              # Accede a las claves con os.environ.get
import time                            # Permite pausas controladas para no saturar APIs

# --- 2. EXTRACCIÓN Y MANEJO DE APIS ---
# Comunicación y autenticación con APIs de música
import requests                        # Peticiones HTTP genéricas (ej. Last.fm)
import spotipy                          # Wrapper para la API de Spotify
from spotipy.oauth2 import SpotifyClientCredentials  # OAuth seguro de Spotify
from spotipy.exceptions import SpotifyException      # Manejo de errores de Spotify (ej. 429)

# --- 3. ESTRUCTURA Y TRANSFORMACIÓN DE DATOS ---
import pandas as pd                     # DataFrames para limpiar, unir y manipular datos

# --- 4. NORMALIZACIÓN Y UTILIDADES ---
from urllib.parse import quote           # Codifica URLs para nombres con espacios/acentos

In [12]:
# Conexión a sql
import mysql.connector
from mysql.connector import Error # Manejo de errores específicos de MySQL

# --- Otras Utilidades ---
import numpy as np # Necesario para manejar np.nan (valores nulos) al insertar DataFrames

In [ ]:
# --- COMPROBACIÓN Y CONEXIÓN CON SPOTIFY ---
# Recupera las credenciales del archivo .env
client_id = os.getenv('SPOTIFY_CLIENT_ID')
client_secret = os.getenv('SPOTIFY_CLIENT_SECRET')

# Verifica que ambas credenciales estén disponibles
if not client_id or not client_secret:
    print('⚠️ ADVERTENCIA: Las credenciales de Spotify no están cargadas. Revisa tu archivo .env.')
else:
    # Configura la autenticación segura con Spotify usando OAuth de aplicación
    mis_credenciales = SpotifyClientCredentials(
        client_id=client_id,
        client_secret=client_secret
    )
    # Inicializa el cliente de Spotify con las credenciales
    spotify = spotipy.Spotify(auth_manager=mis_credenciales)
    print('✅ Conexión con Spotify inicializada.')

✅ Conexión con Spotify inicializada.


In [ ]:
# --- PRUEBA DE CONEXIÓN CON SPOTIFY ---
# Objetivo: comprobar que el cliente de Spotify funciona y devuelve datos válidos.

# Realizamos una búsqueda de artistas por género (genre solo funciona con artistas)
prueba = spotify.search(
    q='genre:country',   # Buscamos artistas del género "country"
    type='artist',     # Tipo de búsqueda: artistas
    limit=1            # Solo 1 resultado, suficiente para probar
)

# Extraemos el primer artista de los resultados
artista = prueba['artists']['items'][0]

# Mostramos el nombre del artista para confirmar que la conexión y la búsqueda funcionan
print('✅ Conexión exitosa con Spotify')
print('🎵 Artista de prueba encontrado:', artista['name'])

✅ Conexión exitosa con Spotify
🎵 Artista de prueba encontrado: Teddy Swims


In [16]:
# --- COMPROBACIÓN DE LA CLAVE DE LAST.FM ---
# Recupera la API key desde el archivo .env
LASTFM_API_KEY = os.getenv('LASTFM_API_KEY')

# Verifica que la clave exista antes de usarla
if not LASTFM_API_KEY:
    print('⚠️ ADVERTENCIA: La clave de Last.fm no está cargada. Revisa tu archivo .env.')
else:
    # Confirmamos que la clave está disponible para futuras peticiones
    print('✅ Conexión con Last.fm inicializada.')

✅ Conexión con Last.fm inicializada.


In [18]:
# --- PRUEBA DE CONEXIÓN BÁSICA CON LAST.FM ---
if not LASTFM_API_KEY:
    print('❌ Clave API de Last.fm no definida.')
else:
    try:
        # Hacemos una petición mínima para verificar la conexión usando el método 'chart.getTopArtists'
        respuesta = requests.get(
            'http://ws.audioscrobbler.com/2.0/',
            params={
                'method': 'chart.gettopartists',
                'api_key': LASTFM_API_KEY,
                'format': 'json',
                'limit': 1  # Solo necesitamos 1 artista para probar
            }
        )
        respuesta.raise_for_status()
        print('✅ Conexión exitosa con Last.fm')
    except requests.exceptions.RequestException as e:
        print(f'❌ Error al conectar con Last.fm: {e}')

✅ Conexión exitosa con Last.fm


## Buscar canciones

### Spotify

In [22]:
# ────────────────────────────────────────────────
# FUNCIÓN SEGURA PARA HACER SEARCH
# ────────────────────────────────────────────────
def spotify_search_seguro(spotify, query, tipo, limite=10, reintentos=3):
    """
    Función que busca en Spotify de forma segura.
    Detecta el error 429 (rate limit) y espera si es necesario.
    
    Parámetros:
    - spotify: objeto de la API
    - query: texto a buscar
    - tipo: "artist" o "album"
    - limite: máximo resultados
    - reintentos: cuántas veces intentar si hay error 429
    """
    intento = 0  # Contador de intentos

    while intento < reintentos:
        try:
            # Llamada a Spotify
            return spotify.search(q=query, type=tipo, limit=limite)

        except SpotifyException as e:
            if e.http_status == 429:
                # Si nos dicen "espera", tomamos el tiempo de Retry-After
                espera = int(e.headers.get("Retry-After", 1))
                print(f"⏳ Demasiadas peticiones. Esperando {espera} segundos...")
                time.sleep(espera)
                intento += 1  # Sumamos intento
            else:
                print("❌ Error de Spotify:", e)
                return None

        except Exception as e:
            # Capturamos cualquier error inesperado
            print("❌ Error inesperado:", e)
            return None

    # Si agotamos todos los intentos
    print("⚠️ Demasiados intentos. Se omite esta búsqueda.")
    return None

In [23]:
# ────────────────────────────────────────────────
# FUNCIÓN SEGURA PARA OBTENER CANCIONES DE UN ÁLBUM
# ────────────────────────────────────────────────
def spotify_album_tracks_seguro(spotify, id_album, reintentos=3):
    """
    Función que obtiene las canciones de un álbum de forma segura.
    Maneja errores 429 y cualquier otro error sin que el programa se caiga.
    """
    intento = 0

    while intento < reintentos:
        try:
            return spotify.album_tracks(id_album)

        except SpotifyException as e:
            if e.http_status == 429:
                espera = int(e.headers.get("Retry-After", 1))
                print(f"⏳ Rate limit en álbum. Esperando {espera} segundos...")
                time.sleep(espera)
                intento += 1
            else:
                print("❌ Error de Spotify:", e)
                return None

        except Exception as e:
            print("❌ Error inesperado:", e)
            return None

    print("⚠️ Demasiados intentos con este álbum.")
    return None

In [24]:
# ────────────────────────────────────────────────
# FUNCIONES DE NEGOCIO (LÓGICA DEL PROBLEMA)
# ────────────────────────────────────────────────

def buscar_artistas_por_genero(spotify, genero):
    """
    Busca artistas de un género específico.
    """
    resultado = spotify_search_seguro(spotify, "genre:" + genero, "artist")
    if resultado is None:
        return []
    return resultado["artists"]["items"]  # Devolvemos la lista de artistas

In [25]:
def buscar_albumes_por_artista_y_año(spotify, nombre_artista, año):
    """
    Busca álbumes de un artista en un año concreto.
    """
    query = "artist:" + nombre_artista + " year:" + str(año)
    resultado = spotify_search_seguro(spotify, query, "album")
    if resultado is None:
        return []
    return resultado["albums"]["items"]  # Devolvemos solo los álbumes

In [26]:
def obtener_canciones_de_album(spotify, id_album):
    """
    Devuelve las canciones de un álbum usando la función segura.
    """
    resultado = spotify_album_tracks_seguro(spotify, id_album)
    if resultado is None:
        return []
    return resultado["items"]

In [27]:
# ────────────────────────────────────────────────
# PROCESAR TODAS LAS CANCIONES DE UN GÉNERO
# ────────────────────────────────────────────────
def procesar_genero(spotify, genero, año, albumes_ya_vistos):
    """
    Devuelve una lista de canciones de un género, sin duplicados de álbumes.
    """
    canciones = []

    artistas = buscar_artistas_por_genero(spotify, genero)
    for artista in artistas:
        nombre_artista = artista["name"]

        albumes = buscar_albumes_por_artista_y_año(spotify, nombre_artista, año)
        for album in albumes:
            id_album = album["id"]
            nombre_album = album["name"]

            # Evitamos duplicados
            if id_album in albumes_ya_vistos:
                continue
            albumes_ya_vistos.add(id_album)

            canciones_album = obtener_canciones_de_album(spotify, id_album)
            for cancion in canciones_album:
                # Guardamos info clara
                canciones.append({
                    "nombre": cancion["name"],
                    "artista": nombre_artista,
                    "album": nombre_album,
                    "genero": genero,
                    "año": año
                })

    return canciones

In [28]:
# ────────────────────────────────────────────────
# MOSTRAR RESUMEN FINAL
# ────────────────────────────────────────────────
def mostrar_resumen(canciones, generos):
    print("RESUMEN:")
    for genero in generos:
        contador = sum(1 for c in canciones if c["genero"] == genero)
        print(f"- {genero}: {contador} canciones")

In [33]:
# ────────────────────────────────────────────────
# FUNCIÓN PRINCIPAL
# ────────────────────────────────────────────────
def main(spotify, año):
    """
    Función principal que orquesta todo el flujo.
    """
    generos = ["country", "latin", "jazz", "rock"]
    año = 2012
    todas_las_canciones = []
    albumes_ya_vistos = set()

    print("Buscando canciones del año", año)
    print()

    for genero in generos:
        print("Género:", genero)
        canciones_genero = procesar_genero(spotify, genero, año, albumes_ya_vistos)
        todas_las_canciones.extend(canciones_genero)
        print("  → Canciones de este género añadidas\n")

    mostrar_resumen(todas_las_canciones, generos)
    print("Total de canciones encontradas:", len(todas_las_canciones))

    # --- Crear DataFrame directamente dentro de la función ---
    if todas_las_canciones:
        df = pd.DataFrame(todas_las_canciones)
        print("\nPrimeras 5 canciones:")
        print(df.head())
        # Guardar a CSV opcional
        df.to_csv(f"canciones_{año}.csv", index=False)
        print(f"\n✅ DataFrame guardado como canciones_{año}.csv")
        return df
    else:
        print("❌ No se encontraron canciones.")
        return None

In [34]:
# Llamamos a la función principal para obtener el DataFrame
todas_las_canciones_df = main(spotify, 2012)

Buscando canciones del año 2012

Género: country
  → Canciones de este género añadidas

Género: latin
  → Canciones de este género añadidas

Género: jazz
  → Canciones de este género añadidas

Género: rock
  → Canciones de este género añadidas

RESUMEN:
- country: 489 canciones
- latin: 212 canciones
- jazz: 1033 canciones
- rock: 514 canciones
Total de canciones encontradas: 2248

Primeras 5 canciones:
               nombre     artista  \
0  Black Bottom Stomp  Jelly Roll   
1         Hyena Stomp  Jelly Roll   
2    Blue Blood Blues  Jelly Roll   
3   Smoke House Blues  Jelly Roll   
4    Winin' Boy Blues  Jelly Roll   

                                            album   genero   año  
0  Best of the Essential Years: Jelly Roll Morton  country  2012  
1  Best of the Essential Years: Jelly Roll Morton  country  2012  
2  Best of the Essential Years: Jelly Roll Morton  country  2012  
3  Best of the Essential Years: Jelly Roll Morton  country  2012  
4  Best of the Essential Years: Jel

### LASTFM

In [35]:
# ────────────────────────────────────────────────
# FUNCIÓN PARA CONSULTAR INFO DE ARTISTA EN LAST.FM
# ────────────────────────────────────────────────
def busqueda_info_artista(artista, api_key):
    """
    Consulta resumen y oyentes de un artista en Last.fm
    """
    url = "http://ws.audioscrobbler.com/2.0/"
    params = {
        "method": "artist.getinfo",
        "artist": quote(artista),
        "api_key": api_key,
        "format": "json"
    }

    try:
        resp = requests.get(url, params=params, timeout=10)
        resp.raise_for_status()
        data = resp.json()

        if "artist" in data:
            bio = data["artist"].get("bio", {}).get("summary", "").split("<a href")[0].strip()
            listeners = int(data["artist"].get("stats", {}).get("listeners", 0))
            return {"artista": artista, "bio_resumen": bio, "listeners": listeners, "consulta_exitosa": True}
        else:
            return {"artista": artista, "consulta_exitosa": False, "error_lastfm": "No encontrado"}

    except Exception as e:
        return {"artista": artista, "consulta_exitosa": False, "error_lastfm": str(e)}

In [36]:
# ────────────────────────────────────────────────
# GENERAR DATAFRAME DE ARTISTAS
# ────────────────────────────────────────────────
def generar_datos_lastfm(canciones_df, api_key):
    if not api_key:
        print("❌ ERROR: La clave de Last.fm no está configurada")
        return None

    artistas_unicos = canciones_df['artista'].unique()
    resultados = []

    # Bucle simple sobre cada artista
    for artista in artistas_unicos:
        print(f"Consultando {artista}...")
        info = busqueda_info_artista(artista, api_key)
        resultados.append(info)

    # Convertir a DataFrame
    df_lastfm = pd.DataFrame(resultados)
    return df_lastfm

# Supongamos que tienes tu DataFrame de Spotify
# todas_las_canciones_df

LASTFM_API_KEY = os.getenv("LASTFM_API_KEY") # Aseguramos que se usa la clave cargada

df_lastfm = generar_datos_lastfm(todas_las_canciones_df, LASTFM_API_KEY)

print(df_lastfm.head())

Consultando Jelly Roll...
Consultando Johnny Cash...
Consultando ROSALÍA...
Consultando Beéle...
Consultando Quevedo...
Consultando Mora...
Consultando Frank Sinatra...
Consultando Nat King Cole...
Consultando Louis Armstrong...
Consultando Robbie Williams...
Consultando Earth, Wind & Fire...
Consultando Ella Fitzgerald...
Consultando Dean Martin...
Consultando Norah Jones...
Consultando Bing Crosby...
Consultando Genki Rockets...
Consultando Linkin Park...
Consultando Arctic Monkeys...
Consultando The Beatles...
Consultando Queen...
Consultando Imagine Dragons...
Consultando Maroon 5...
Consultando Elton John...
       artista                                        bio_resumen  listeners  \
0   Jelly Roll  Jason Bradley DeFord (born December 4, 1984 in...     311162   
1  Johnny Cash  Johnny Cash (born J.R. Cash; February 26, 1932...    4043320   
2      ROSALÍA  Rosalia Vila Tobella, known mononymously as RO...    1741609   
3        Beéle  Brandon De Jesús López Orozco, mejor conoci